In [15]:
import pandas as pd
import random as rnd

rows = 500
df = pd.read_csv("../Datasets/discharge.csv", nrows=rows)
df.to_json('../Datasets/test.json')

In [22]:
random = rnd.randint(1,rows)
test_cell = df.iloc[random, 7]
print(f"{random}, {test_cell}")

349,  
Name:  ___                Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.
 
Chief Complaint:
Altered mental status, leukocytosis
 
Major Surgical or Invasive Procedure:
NONE

 
History of Present Illness:
___ PMH cognitive impairment d/t cerebral aneurysm rupture ___, 
IgA-kappa myeloma on C2 bortez/lenalidomide with daily 
dexamethasone, being treated for C. diff PO vanc 500mg qid at 
___ for leukocytosis (14.7), worsening anemia (24.6), 
and frank hematochezia x1. 
  The patient denies abdominal pain, diarrhea, f/c, n/v, CP, 
dyspnea, syncope, confusion. 
  Of note, she was admitted ___ for AMS/UTI (Ecoli, 
Proteus) -- treated with 5d ceftriaxone at the time; ___ UCx 
showed: E.coli resistant to ampicillin, unasyn, Bactrim. Per 
last d/c summary, Cipro avoided d/t potential for causing AMS. 
Per last d/c summary dau